
cied2000?


In [ ]:
#include <math.h>

double deg2Rad(const double deg)
{
	return (deg * (M_PI / 180.0));
}


// source: https://github.com/gfiumara/CIEDE2000/blob/master/CIEDE2000.cpp
double CIEDE2000(double l1, double a1, double b1, double l2, double a2, double b2)
{
	/* 
	 * "For these and all other numerical/graphical 􏰀delta E00 values
	 * reported in this article, we set the parametric weighting factors
	 * to unity(i.e., k_L = k_C = k_H = 1.0)." (Page 27).
	 */
	const double k_L = 1.0, k_C = 1.0, k_H = 1.0;
	const double deg360InRad = deg2Rad(360.0);
	const double deg180InRad = deg2Rad(180.0);
	const double pow25To7 = 6103515625.0; /* pow(25, 7) */
	
	/*
	 * Step 1 
	 */
	/* Equation 2 */
	double C1 = sqrt((a1 * a1) + (b1 * b1));
	double C2 = sqrt((a2 * a2) + (b2 * b2));
	/* Equation 3 */
	double barC = (C1 + C2) / 2.0;
	/* Equation 4 */
	double G = 0.5 * (1 - sqrt(pow(barC, 7) / (pow(barC, 7) + pow25To7)));
	/* Equation 5 */
	double a1Prime = (1.0 + G) * a1;
	double a2Prime = (1.0 + G) * a2;
	/* Equation 6 */
	double CPrime1 = sqrt((a1Prime * a1Prime) + (b1 * b1));
	double CPrime2 = sqrt((a2Prime * a2Prime) + (b2 * b2));
	/* Equation 7 */
	double hPrime1;
	if (b1 == 0 && a1Prime == 0)
		hPrime1 = 0.0;
	else {
		hPrime1 = atan2(b1, a1Prime);
		/* 
		 * This must be converted to a hue angle in degrees between 0 
		 * and 360 by addition of 2􏰏 to negative hue angles.
		 */
		if (hPrime1 < 0)
			hPrime1 += deg360InRad;
	}
	double hPrime2;
	if (b2 == 0 && a2Prime == 0)
		hPrime2 = 0.0;
	else {
		hPrime2 = atan2(b2, a2Prime);
		/* 
		 * This must be converted to a hue angle in degrees between 0 
		 * and 360 by addition of 2􏰏 to negative hue angles.
		 */
		if (hPrime2 < 0)
			hPrime2 += deg360InRad;
	}
	
	/*
	 * Step 2
	 */
	/* Equation 8 */
	double deltaLPrime = l2 - l1;
	/* Equation 9 */
	double deltaCPrime = CPrime2 - CPrime1;
	/* Equation 10 */
	double deltahPrime;
	double CPrimeProduct = CPrime1 * CPrime2;
	if (CPrimeProduct == 0)
		deltahPrime = 0;
	else {
		/* Avoid the fabs() call */
		deltahPrime = hPrime2 - hPrime1;
		if (deltahPrime < -deg180InRad)
			deltahPrime += deg360InRad;
		else if (deltahPrime > deg180InRad)
			deltahPrime -= deg360InRad;
	}
	/* Equation 11 */
	double deltaHPrime = 2.0 * sqrt(CPrimeProduct) *
    sin(deltahPrime / 2.0);
	
	/*
	 * Step 3
	 */
	/* Equation 12 */
	double barLPrime = (l1 + l2) / 2.0;
	/* Equation 13 */
	double barCPrime = (CPrime1 + CPrime2) / 2.0;
	/* Equation 14 */
	double barhPrime, hPrimeSum = hPrime1 + hPrime2;
	if (CPrime1 * CPrime2 == 0) {
		barhPrime = hPrimeSum;
	} else {
		if (fabs(hPrime1 - hPrime2) <= deg180InRad)
			barhPrime = hPrimeSum / 2.0;
		else {
			if (hPrimeSum < deg360InRad)
				barhPrime = (hPrimeSum + deg360InRad) / 2.0;
			else
				barhPrime = (hPrimeSum - deg360InRad) / 2.0;
		}
	}
	/* Equation 15 */
	double T = 1.0 - (0.17 * cos(barhPrime - deg2Rad(30.0))) +
    (0.24 * cos(2.0 * barhPrime)) +
    (0.32 * cos((3.0 * barhPrime) + deg2Rad(6.0))) - 
    (0.20 * cos((4.0 * barhPrime) - deg2Rad(63.0)));
	/* Equation 16 */
	double deltaTheta = deg2Rad(30.0) *
    exp(-pow((barhPrime - deg2Rad(275.0)) / deg2Rad(25.0), 2.0));
	/* Equation 17 */
	double R_C = 2.0 * sqrt(pow(barCPrime, 7.0) /
    (pow(barCPrime, 7.0) + pow25To7));
	/* Equation 18 */
	double S_L = 1 + ((0.015 * pow(barLPrime - 50.0, 2.0)) /
    sqrt(20 + pow(barLPrime - 50.0, 2.0)));
	/* Equation 19 */
	double S_C = 1 + (0.045 * barCPrime);
	/* Equation 20 */
	double S_H = 1 + (0.015 * barCPrime * T);
	/* Equation 21 */
	double R_T = (-sin(2.0 * deltaTheta)) * R_C;
	
	/* Equation 22 */
  double deltaE = sqrt(
    pow(deltaLPrime / (k_L * S_L), 2.0) +
    pow(deltaCPrime / (k_C * S_C), 2.0) +
    pow(deltaHPrime / (k_H * S_H), 2.0) + 
    (R_T * (deltaCPrime / (k_C * S_C)) * (deltaHPrime / (k_H * S_H))));

	return (deltaE);
}



rgb 2 lab?


In [ ]:
#include <math.h>

double F(double input) // function f(...), which is used for defining L, a and b changes within [4/29,1]
{
  if (input > 0.008856)
    return (pow(input, 0.333333333)); // maximum 1
  else
    return ((841/108)*input + 4/29);  //841/108 = 29*29/36*16
}


void XYZtoLab(double X, double Y, double Z, double *L, double *a, double *b)
{
  // TODO: make sure these are correct
  const double Xo = 244.66128; // reference white
  const double Yo = 255.0;
  const double Zo = 277.63227;
  *L = 116 * F(Y / Yo) - 16; // maximum L = 100
  *a = 500 * (F(X / Xo) - F(Y / Yo)); // maximum 
  *b = 200 * (F(Y / Yo) - F(Z / Zo));
}


// source http://www.easyrgb.com/en/math.php
void RGBtoXYZ(double R, double G, double B, double *X, double *Y, double *Z) {
  // Assume RGB has the type invariance satisfied, i.e., channels \in [0,255]
  float var_R = R / 255.0;
  float var_G = G / 255.0;
  float var_B = B / 255.0;

  var_R = (var_R > 0.04045) ? pow((var_R + 0.055) / 1.055, 2.4)
                            : var_R / 12.92;
  var_G = (var_G > 0.04045) ? pow((var_G + 0.055) / 1.055, 2.4)
                            : var_G / 12.92;
  var_B = (var_B > 0.04045) ? pow((var_B + 0.055) / 1.055, 2.4)
                            : var_B / 12.92;

  var_R *= 100;
  var_G *= 100;
  var_B *= 100;

  *X = var_R * 0.4124 + var_G * 0.3576 +
       var_B * 0.1805;
  *Y = var_R * 0.2126 + var_G * 0.7152 +
       var_B * 0.0722;
  *Z = var_R * 0.0193 + var_G * 0.1192 +
       var_B * 0.9505;
}


double *rgb2lab(int R, int G, int B){
  static double Lab[3] = {0, 0, 0};
  double X, Y, Z;
  RGBtoXYZ(R, G, B, &X, &Y, &Z);
  XYZtoLab(X, Y, Z, &Lab[0], &Lab[1], &Lab[2]);
  return Lab;
}





try cied2000?


In [ ]:
import assert from 'assert'
import colors from 'color-name'

async function testCIED2000() {
  const {rgb2lab} = await importer.import('rgb 2 lab')
  let [R, G, B] = colors.red
  let [R2, G2, B2] = colors.darkred
  let [l, a, b] = rgb2lab(3, R, G, B)
  let [l2, a2, b2] = rgb2lab(3, R2, G2, B2)
  const {CIEDE2000} = await importer.import('cied2000')
  let ciede = CIEDE2000(1, l, a, b, l2, a2, b2)
  console.log(ciede)
  assert.ok(ciede < 20, 'colors are the same')
  return ciede
}

export default testCIED2000
